## 一、配置环境变量

In [1]:
import os
from dotenv import dotenv_values

In [2]:
env_variables = dotenv_values('.env')
for var in env_variables:
    print(var)
    os.environ[var] = env_variables[var]

ZHIPUAI_API_KEY
DASHSCOPE_API_KEY


In [3]:
os.getenv("DASHSCOPE_API_KEY")

'sk-f7e5ff2257cd4cff95eaedffb9ddb35e'

## 二、配置LLM

In [4]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import os
from typing import Dict, List, Optional, Tuple, Union

from zhipuai import ZhipuAI
import dashscope
import random
from http import HTTPStatus
import dashscope
from dashscope import Generation  # 建议dashscope SDK 的版本 >= 1.14.0

PROMPT_TEMPLATE = dict(
    RAG_PROMPT_TEMPALTE="""使用以上下文来回答用户的问题。如果你不知道答案，就说你不知道。总是使用中文回答。
        问题: {question}
        可参考的上下文：
        ···
        {context}
        ···
        如果给定的上下文无法让你做出回答，请回答数据库中没有这个内容，你不知道。
        有用的回答:""",
    InternLM_PROMPT_TEMPALTE="""先对上下文进行内容总结,再使用上下文来回答用户的问题。如果你不知道答案，就说你不知道。总是使用中文回答。
        问题: {question}
        可参考的上下文：
        ···
        {context}
        ···
        如果给定的上下文无法让你做出回答，请回答数据库中没有这个内容，你不知道。
        有用的回答:"""
)


class BaseModel:
    def __init__(self, path: str = '') -> None:
        self.path = path

    def chat(self, prompt: str, history: List[dict], content: str) -> str:
        pass

    def load_model(self):
        pass

class GLM4Chat(BaseModel):
    def __init__(self, path: str = '', model: str = "glm-4") -> None:
        super().__init__(path)
        self.model = model

    def chat(self, prompt: str, history: List[dict], content: str) -> str:
        client = ZhipuAI(api_key=os.getenv("ZHIPUAI_API_KEY"))  # 填写您自己的APIKey
        history.append({'role': 'user', 'content': PROMPT_TEMPLATE['RAG_PROMPT_TEMPALTE'].format(question=prompt, context=content)})
        response = client.chat.completions.create(
            model="glm-4",  # 填写需要调用的模型名称
            messages=history
        )
        return response.choices[0].message

class QwenChat(BaseModel):
    def __init__(self, path: str = '', model: str = "qwen-turbo") -> None:
        super().__init__(path)
        dashscope.api_key = os.getenv("DASHSCOPE_API_KEY")
        self.model = model

    def chat(self, prompt: str, history: List[dict], content: str) -> str:
        history.append({'role': 'user', 'content': PROMPT_TEMPLATE['RAG_PROMPT_TEMPALTE'].format(question=prompt, context=content)})
        response = Generation.call(model="qwen-turbo",
                                messages=history,
                                # 设置随机数种子seed，如果没有设置，则随机数种子默认为1234
                                seed=random.randint(1, 10000),
                                # 将输出设置为"message"格式
                                result_format='message')
        if response.status_code == HTTPStatus.OK:
            return response.output.choices[0].message["content"]
        else:
            print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
                response.request_id, response.status_code,
                response.code, response.message
            ))


class OpenAIChat(BaseModel):
    def __init__(self, path: str = '', model: str = "gpt-3.5-turbo-1106") -> None:
        super().__init__(path)
        self.model = model

    def chat(self, prompt: str, history: List[dict], content: str) -> str:
        from openai import OpenAI
        client = OpenAI()   
        history.append({'role': 'user', 'content': PROMPT_TEMPLATE['RAG_PROMPT_TEMPALTE'].format(question=prompt, context=content)})
        response = client.chat.completions.create(
            model=self.model,
            messages=history,
            max_tokens=150,
            temperature=0.1
        )
        return response.choices[0].message.content

class InternLMChat(BaseModel):
    def __init__(self, path: str = '') -> None:
        super().__init__(path)
        self.load_model()

    def chat(self, prompt: str, history: List = [], content: str='') -> str:
        prompt = PROMPT_TEMPLATE['InternLM_PROMPT_TEMPALTE'].format(question=prompt, context=content)
        response, history = self.model.chat(self.tokenizer, prompt, history)
        return response


    def load_model(self):
        import torch
        from transformers import AutoTokenizer, AutoModelForCausalLM
        self.tokenizer = AutoTokenizer.from_pretrained(self.path, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(self.path, torch_dtype=torch.float16, trust_remote_code=True).cuda()


In [5]:
qwenchat = QwenChat()
qwenchat.chat(history=[], prompt="hello", content="")

'你好！有什么我可以帮助你的吗？'

## 三、设置知识库FaissVectorStore

In [6]:
import os
import time
from typing import Dict, List, Optional, Tuple, Union
import json

import faiss

from RAG.Embeddings import BaseEmbeddings, OpenAIEmbedding, JinaEmbedding, ZhipuEmbedding
import numpy as np
from tqdm import tqdm

from RAG.VectorBase import VectorStore


class FaissVectorStore(VectorStore):

    def __init__(self, document: List[str] = ['']) -> None:
        super().__init__(document)
        self.document = document

    def get_vector(self, EmbeddingModel: BaseEmbeddings) -> List[List[float]]:
        self.vectors = []
        for doc in tqdm(self.document, desc="Calculating embeddings"):
            self.vectors.append(EmbeddingModel.get_embedding(doc))
        return self.vectors

    def persist(self, path: str = 'storage'):
        if not os.path.exists(path):
            os.makedirs(path)
        with open(f"{path}/doecment.json", 'w', encoding='utf-8') as f:
            json.dump(self.document, f, ensure_ascii=False)
        if self.vectors:
            with open(f"{path}/vectors.json", 'w', encoding='utf-8') as f:
                json.dump(self.vectors, f)

    def load_vector(self, path: str = 'storage'):
        with open(f"{path}/vectors.json", 'r', encoding='utf-8') as f:
            self.vectors = json.load(f)
        with open(f"{path}/doecment.json", 'r', encoding='utf-8') as f:
            self.document = json.load(f)

    def get_similarity(self, vector1: List[float], vector2: List[float]) -> float:
        return BaseEmbeddings.cosine_similarity(vector1, vector2)

    def generateIndexFlatL2(self):
        dim, measure = 768, faiss.METRIC_L2
        param = 'Flat'
        vectors = np.array([vector for vector in self.vectors]).astype('float32')
        index = faiss.index_factory(dim, param, measure)
        index.add(vectors)  # 将向量库中的向量加入到index中
        return index


    def query(self, query: str, EmbeddingModel: BaseEmbeddings, k: int = 1) -> List[str]:

        start_time = time.time()
        index = self.generateIndexFlatL2()
        end_time = time.time()
        print(' 构建索引 cost %f second' % (end_time - start_time))
        query_vector = np.array([EmbeddingModel.get_embedding(query)]).astype("float32")

        end_time = time.time()
        D, I = index.search(query_vector, k)  # xq为待检索向量，返回的I为每个待检索query最相似TopK的索引list，D为其对应的距离
        print(' 检索 cost %f second' % (time.time() - end_time))
        return np.array(self.document)[I[0,:]].tolist()

In [7]:
vector = FaissVectorStore()

In [8]:
vector.load_vector('../../storage/history_24_1') # 加载本地的数据库

In [9]:
len(vector.vectors[0])

768

## 四、定义embedding算法

In [ ]:
from RAG.PaddleEmbedding import PaddleEmbedding
embedding = PaddleEmbedding() # 创建EmbeddingModel

In [ ]:
question = '介绍一下秦二世'
content = vector.query(question, EmbeddingModel=embedding, k=3)[0]
print(f'知识库输出：{content}')
chat = QwenChat()
print(chat.chat(question, [], content))